<a href="https://colab.research.google.com/github/AyanChattaraj/geb-ai-demo/blob/main/r_2_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install -qU langchain-google-genai langchain_mistralai mistralai

In [2]:
import getpass
import os

if "GOOGLE_API_KEY" not in os.environ:
    os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter your Google AI API key: ")

if "MISTRAL_API_KEY" not in os.environ:
    os.environ["MISTRAL_API_KEY"] = getpass.getpass("Enter your Mistral AI API key: ")

Enter your Google AI API key: ··········
Enter your Mistral AI API key: ··········


In [4]:
from langchain.llms.base import LLM

class CodestralLLM(LLM):
    def __init__(self, api_key, model_name="codestral-latest"):
        from mistralai import Mistral
        self.client = Mistral(api_key=api_key)
        self.model = model_name

    def _call(self, prompt, stop=None):
        response = self.client.completions.create(
            model=self.model,
            prompt=prompt,
            stop=stop,
            temperature=0,
        )
        return response.choices[0].text

In [21]:
import re
import os
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain import PromptTemplate, LLMChain
from mistralai import Mistral
from langchain_mistralai import ChatMistralAI

class RToPythonConverter:
    def __init__(self, model="gemini-2.0-flash-001", temperature=0):
        """Initialize the Gemini AI model."""
        self.llm = ChatGoogleGenerativeAI(
            model=model,
            temperature=temperature,
            max_tokens=None,
            timeout=None,
            max_retries=2,
        )
        self.template = """You are an expert AI programmer skilled in multiple languages.
Your task is to analyze the provided **R** program, understand its logic, functions, and dependencies, then generate an equivalent **Python** program.
Follow these rules:
1. Identify the core logic of the R program and translate it **accurately** into Python.
2. Replace **R-specific functions** with the appropriate Python equivalents.
3. Ensure external libraries used in R (e.g., `ggplot2`, `dplyr`) are mapped to suitable Python packages (e.g., `matplotlib`, `pandas`).
4. Maintain the readability and efficiency of the translated Python code.
5. If an R function has no direct Python equivalent, construct the logic manually.
6. 1st para of Python program should briefly summarize the code as docstring
R Code:
{r_code}

Python Code:
 """
        self.prompt = PromptTemplate(input_variables=["r_code"], template=self.template)
        self.llm_chain = LLMChain(prompt=self.prompt, llm=self.llm)

    def convert_r_to_python(self, r_code):
        """Convert R code to Python using Gemini AI."""
        response = self.llm_chain.run(r_code)

        # Extract Python code within ```python ... ```
        match = re.search(r"```python(.*?)```", response, re.DOTALL)
        clean_response = match.group(1).strip() if match else response.strip()

        return clean_response

    def save_python_code(self, python_code, output_file):
        """Save generated Python code to a file."""
        output_dir = os.path.dirname(output_file)
        os.makedirs(output_dir, exist_ok=True)

        with open(output_file, "w") as file:
            file.write(python_code)

        print(f"\n✅ Python code has been saved to: {output_file}")

    def convert_r_file_to_python(self, r_file_path, output_file):
        """Read an R program from file, convert it to Python, and save the result."""
        if not os.path.exists(r_file_path):
            print(f"❌ Error: File '{r_file_path}' not found.")
            return

        # Read the R program file
        with open(r_file_path, "r") as file:
            r_code = file.read()

        # Convert the R code to Python
        python_code = self.convert_r_to_python(r_code)

        # Save the generated Python code to file
        self.save_python_code(python_code, output_file)

    def convert_and_save(self, r_file_path, output_file, model_config):
      """
        Converts R code to Python using specified model and saves to a folder.
      """
      model_name = model_config["model_name"]
      model_params = model_config.get("model_params", {})  # Get params if present

      # Create model instance based on model_name
      if model_name == "gemini-2.0-flash-001":
        self.llm = ChatGoogleGenerativeAI(
              model=model_name,
              **model_params)
      elif model_name == "codestral-mamba-latest":
        self.llm = ChatMistralAI(
              model=model_name,
              **model_params
                )
      else:
          raise ValueError(f"Unsupported model: {model_name}")

      # Update LLMChain with the new model
      self.llm_chain = LLMChain(prompt=self.prompt, llm=self.llm)


      self.convert_r_file_to_python(r_file_path, output_file)

    def convert_and_save_all(self, r_directory, output_dir, model_config):
      """
      Converts all R files in a directory to Python and saves them.
      """
      for filename in os.listdir(r_directory):
          if filename.endswith(".r"):  # Process only .r files
              r_file_path = os.path.join(r_directory, filename)
              # Create output file name with .py extension
              output_file = os.path.join(output_dir, model_config["model_name"], filename[:-2] + ".py")
              self.convert_and_save(r_file_path, output_file, model_config)  # Use existing method


# Test the reusable method with a sample R program file
if __name__ == "__main__":
    r_directory = "/content/Input"  # Input directory containing R files
    output_base_dir = "/content/Output"

    model_configs = [
        {"model_name": "gemini-2.0-flash-001",
         "model_params":
          {
              "temperature": 0,
              "max_tokens":None,
              "timeout":None,
              "max_retries":2,
              "api_key":os.environ.get("GOOGLE_API_KEY")
              }
        },

        {"model_name": "codestral-mamba-latest",
         "model_params":
          {
              "temperature": 0,
              "max_retries":2,
              "api_key":os.environ.get("MISTRAL_API_KEY")
              }
         },
    ]

    converter = RToPythonConverter()

    for config in model_configs:
        converter.convert_and_save_all(r_directory, output_base_dir, config)



✅ Python code has been saved to: /content/Output/gemini-2.0-flash-001/regression.py

✅ Python code has been saved to: /content/Output/gemini-2.0-flash-001/box_plot.py

✅ Python code has been saved to: /content/Output/gemini-2.0-flash-001/data_manipulation.py

✅ Python code has been saved to: /content/Output/codestral-mamba-latest/regression.py

✅ Python code has been saved to: /content/Output/codestral-mamba-latest/box_plot.py

✅ Python code has been saved to: /content/Output/codestral-mamba-latest/data_manipulation.py


In [20]:
# prompt: delete Output dir

import shutil
import os

# Define the directory to be deleted
dir_path = "/content/Output"

# Check if the directory exists before attempting to delete it
if os.path.exists(dir_path):
  try:
    shutil.rmtree(dir_path)
    print(f"Directory '{dir_path}' and its contents have been successfully deleted.")
  except OSError as e:
    print(f"Error: {e}")
else:
  print(f"Directory '{dir_path}' does not exist.")


Directory '/content/Output' and its contents have been successfully deleted.


**Sample output by Mistral**

**Sample output by gemini**